In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from crush_read import *
from crush_plot import *
PATH

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 150

In [ ]:
study = study_outline(PATH)
targets = study_targets(PATH)
crushes = study_data(study)
crushes = modify(crushes)
crushes = calculate(crushes)

In [ ]:
_ = preprocess(crushes, targets)
c = binary_classes(crushes, drop_cols=False)

Prepare graphs needed for paper.

In [ ]:
c.describe()

All patients will be considered. All protocols should be STOP only.

In [ ]:
print(c['Patient'].unique())

In [ ]:
print(c['Protocol'].unique())

In [ ]:
# Convenience functions
def patient(patient):
    return c.loc[c['Patient'] == patient.upper(), :]

def tissue(tissue):
    return c.loc[c['Tissue'] == tissue.upper(), :]

def protocol(protocol):
    return c.loc[c['Protocol'] == protocol.upper(), :]

def load(load):
    return c.loc[c['Load'] == load.lower(), :]

In [ ]:
sb = tissue('sb')
sb.shape

In [ ]:
colon = tissue('colon')
colon.shape

# COLON PERCENTAGE PLOT

In [ ]:
damage = colon['Trauma Score']
colon_dam = pd.DataFrame({'No Damage': (damage == 0),
                          'Minor Damage': (damage == 1),
                          'Major Damage': (damage == 2)})
colon_dam = pd.concat([colon_dam, colon['Load (g)']], axis=1)
colon_dam.head()

In [ ]:
group = colon_dam.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()
group

In [ ]:
damage_types = ['No Damage', 'Minor Damage', 'Major Damage']
for damage in damage_types:
    group[damage] = group[damage] / group['Count']
group

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
colors = ListedColormap(['blue', 'orange', 'red'])
width = 50

for i, damage in enumerate(damage_types):
    plt.figure()
    plt.bar(x=group['Load (g)'], height=group[damage], width=width, label=damage, color=colors(i))
    plt.legend()
    plt.ylabel('Percentage of Samples')
    plt.xlabel('Load (g)')
    plt.ylim(0, 1)
    ax = plt.gca()
    ticks = ax.get_yticks()
    ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
colors = ListedColormap(['blue', 'orange', 'red'])
width = 50
shift = [-width, 0, width]
plt.figure()
for i, damage in enumerate(damage_types): 
    if i == 0:
        continue
    x_val = group['Load (g)'] + shift[i]
    plt.bar(x=x_val, height=group[damage], width=width, label=damage, color=colors(i))
plt.legend()
plt.ylabel('Percentage of Samples')
plt.xlabel('Load (g)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
nice_colors = ListedColormap(['orangered', 'orange', 'dodgerblue'])

In [ ]:
def disp_stress(x):
    return np.round(9.81 * x.values / (np.pi * ((5/2) ** 2)), 0)
disp_stress(group['Load (g)'])

In [ ]:
width = 50

In [ ]:
plt.figure()
types = list(reversed(damage_types))
for i, damage in enumerate(types):
    inputs = {'x': disp_stress(group['Load (g)']),
              'height': group[damage],
              'width': width,
              'label': damage + f' [{2 - i}]',
              'color': nice_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Trauma Score - Colon', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Pressure (kPa)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
# Same for serosal thickness change
delta = colon['Significant Serosal Change']
colon_del = pd.DataFrame({'No Change': (delta == False),
                          'Significant Change': (delta == True)})
colon_del = pd.concat([colon_del, colon['Load (g)']], axis=1)

In [ ]:
group = colon_del.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()

In [ ]:
ser_colors = ListedColormap(['orchid', 'turquoise'])

In [ ]:
delta_types = ['No Change', 'Significant Change']
for delta in delta_types:
    group[delta] = group[delta] / group['Count']
types = list(reversed(delta_types))

plt.figure()
for i, damage in enumerate(types):
    inputs = {'x': disp_stress(group['Load (g)']),
              'height': group[damage],
              'width': width,
              'label': damage,
              'color': ser_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Serosal Thickness Change - Colon', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Pressure (kPa)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
group

# SMALL BOWEL PERCENTAGE PLOT

In [ ]:
damage = sb['Trauma Score']
sb_dam = pd.DataFrame({'No Damage': (damage == 0),
                       'Minor Damage': (damage == 1),
                       'Major Damage': (damage == 2)})
sb_dam = pd.concat([sb_dam, sb['Load (g)']], axis=1)
sb_dam.head()

In [ ]:
group = sb_dam.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()
group

In [ ]:
damage_types = ['No Damage', 'Minor Damage', 'Major Damage']
for damage in damage_types:
    group[damage] = group[damage] / group['Count']
group

In [ ]:
plt.figure()
types = list(reversed(damage_types))
for i, damage in enumerate(types):
    inputs = {'x': disp_stress(group['Load (g)']),
              'height': group[damage],
              'width': width,
              'label': damage + f' [{2 - i}]',
              'color': nice_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Trauma Score - Small Bowel', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Pressure (kPa)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

In [ ]:
# Same for serosal thickness change
delta = sb['Significant Serosal Change']
sb_del = pd.DataFrame({'No Change': (delta == False),
                       'Significant Change': (delta == True)})
sb_del = pd.concat([sb_del, sb['Load (g)']], axis=1)
group = sb_del.groupby('Load (g)').sum()
group['Count'] = group.sum(axis=1)
group = group.reset_index()

In [ ]:
delta_types = ['No Change', 'Significant Change']
for delta in delta_types:
    group[delta] = group[delta] / group['Count']
types = list(reversed(delta_types))

plt.figure()
for i, damage in enumerate(types):
    inputs = {'x': disp_stress(group['Load (g)']),
              'height': group[damage],
              'width': width,
              'label': damage,
              'color': ser_colors(i)}
    if i == 1:
        bottom = group[types[i - 1]]
        inputs['bottom'] = bottom
    if i > 1:
        bottom = bottom + group[types[i - 1]]
        inputs['bottom'] = bottom
    plt.bar(**inputs)
plt.figlegend(loc='upper center', ncol=len(damage_types))
plt.title('Serosal Thickness Change - Small Bowel', size=10, weight='bold')
plt.ylabel('Percentage of Samples')
plt.xlabel('Pressure (kPa)')
plt.ylim(0, 1)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in ticks])

# Tissue Average Serosal Thickness Plot

In [ ]:
t_pre = colon['Serosal Thickness (mm)']
t_post = colon['Post Serosal Thickness (mm)']
perc_delta = (t_post - t_pre) / t_pre
colon_thickness = pd.DataFrame({'Percent Delta (COLON)': perc_delta,
                                'Load (g)': colon['Load (g)']})
colon_thickness = colon_thickness.groupby('Load (g)').mean()
colon_thickness

In [ ]:
t_pre = sb['Serosal Thickness (mm)']
t_post = sb['Post Serosal Thickness (mm)']
perc_delta = (t_post - t_pre) / t_pre
sb_thickness = pd.DataFrame({'Percent Delta (SB)': perc_delta,
                             'Load (g)': sb['Load (g)']})
sb_thickness = sb_thickness.groupby('Load (g)').mean()
sb_thickness

In [ ]:
group = pd.concat([sb_thickness, colon_thickness], axis=1)
group = group.reset_index()
group

In [ ]:
tissue_colors = ListedColormap(['oldlace', 'sandybrown'])

In [ ]:
types = ['Percent Delta (SB)', 'Percent Delta (COLON)']
labels = ['Small Bowel', 'Colon']
width = 40
shift = [-20, 20]
plt.figure()
for i, typ in enumerate(types):
    inputs = {'x': disp_stress(group['Load (g)'])  + shift[i],
              'height': group[typ].abs(),
              'width': width,
              'label': labels[i],
              'color': tissue_colors(i)}
    plt.bar(**inputs)
plt.figlegend(loc=(0.5, 0.825), ncol=len(damage_types))
plt.title('Serosal Thickness Percent Deformation', size=10, weight='bold')
plt.ylabel('Percent Deformation')
plt.xlabel('Pressure (kPa)')
plt.ylim(0, 0.6)
ax = plt.gca()
ticks = ax.get_yticks()
_ = ax.set_yticklabels(['-{:.0%}'.format(x) for x in ticks])

# Patient Plots

In [ ]:
score = pd.DataFrame({'Trauma Score': crushes['Trauma Score'],
                      'Tissue': crushes['Tissue'],
                      'Load (g)': crushes['Load (g)'],
                      'Patient': crushes['Patient']})

score_colon = score[score['Tissue'] == 'COLON'].groupby(['Patient', 'Tissue', 'Load (g)']).mean()
score_sb = score[score['Tissue'] == 'SB'].groupby(['Patient', 'Tissue', 'Load (g)']).mean()

In [ ]:
score.groupby(['Patient', 'Tissue', 'Load (g)']).mean()

In [ ]:
def plot_patients(score):
    
    # Get list of patients
    def str_int(s):
        return int(s[2:])
    patients = sorted(score.index.levels[0].unique(), key=str_int)
    
    # Create subplots for each patient
    fig = plt.figure(figsize=(7, 5))
    axes = [None] * 10
    for i in range(len(patients)):
        axes[i] = plt.subplot2grid((len(patients) // 2, 2), (i // 2, i % 2))

    # Fill the subplots
    labels = {'SB': 'Small Bowel', 'COLON': 'Colon'}
    for i, pat in enumerate(patients):
        pat_score = score.loc[pat].reset_index()
        tissue = labels[pat_score['Tissue'].iloc[0]]

        ax = axes[i]
        inputs = {'x': disp_stress(pat_score['Load (g)']),  # + shift[i],
                  'y': pat_score['Trauma Score'],
                  'color': 'indigo'}
        ax.scatter(**inputs)

        ax.set_title(f'Patient {pat}', size=8)
        ax.set_ylim(-0.5, 2.5)
        ax.set_yticks([0, 1, 2])
        ax.set_xlim(0, 700)
        if i > 7:
            ax.set_xlabel('Pressure (kPa)', size=10, fontweight='bold')
            ax.set_xticks([100, 200, 300, 400, 500, 600])
        else:
            ax.set_xticklabels([])
    fig.suptitle(f'Trauma Score - {tissue} Samples', size=10, weight='bold')
    plt.subplots_adjust(hspace=0.5)

In [ ]:
plot_patients(score_colon)

In [ ]:
plot_patients(score_sb)

# Pressure, position, stress-strain plots

In [ ]:
example = random(crushes)

In [ ]:
pressure_plot(example, trim=False)

In [ ]:
position_plot(example, trim=False)

In [ ]:
stress_plot(example, trim=False)